In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
import numpy as np

pickle_in = open("real-X2.pickle","rb")
X = pickle.load(pickle_in)[:4000]

pickle_in = open("real-y2.pickle","rb")
y = pickle.load(pickle_in)[:4000]

X = np.array(X/255.0)
y = np.array(y)

dense_layers = [0] #  0,  1,   2
layer_sizes = [64] # 32, 64, 128
conv_layers = [3]  #  1,  2,   3

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.25)) # added dropout to not overfit

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=3,
                      validation_split=0.3,
                      callbacks=[tensorboard])
            model.save("FALL-64x3-CNN.model")
            
            

3-conv-64-nodes-0-dense-1598032853
Epoch 1/3
88/88 [==============================] - 78s 886ms/step - loss: 0.3930 - accuracy: 0.8118 - val_loss: 0.1542 - val_accuracy: 0.9483
Epoch 2/3
88/88 [==============================] - 79s 896ms/step - loss: 0.1180 - accuracy: 0.9600 - val_loss: 0.1066 - val_accuracy: 0.9583
Epoch 3/3
88/88 [==============================] - 79s 893ms/step - loss: 0.0639 - accuracy: 0.9782 - val_loss: 0.0944 - val_accuracy: 0.9658
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: FALL-64x3-CNN.model/assets
